# IQVIA NLP - Clinical Trials Biomarkers

## API Description
Understanding biomarkers for drug efficacy or disease progression is critical in clinical development, but relevant data is often in unstructured text, such as clinical trial records, and drug labels.  For Data Scientists, ClinicalTrials.gov Biomarkers solves this problem by offering an NLP API that extracts gene/protein biomarkers from specific regions of ClinicalTrials.gov records (Inclusion & exclusion criteria, primary and secondary outcomes).

This simple API quickly identifies gene/disease biomarkers from ClinicalTrials.gov records, for known indications (mapped to MedDRA), from an NCT ID.  Biomarker-disease relationships are provided (e.g. positive, negative); diseases are mapped to MedDRA and mechanism of action data are also extracted.

## Accessing the API
In order to consume this API, you will first need to Request access to the Clinical Trials Biomarkers API 
via this link:
https://api-marketplace.work.iqvia.com/s/communityapi/a085w00000ytPu7AAE/external-api-marketplaceclinicaltrialsbiomarkers .

Please refer to "API Documentation" to learn more about accessing and using the API.

## Notebook Description
This notebook is designed to biomarkers and their related diseases from from ClinicalTrials.gov records. Users could specify records of interest by supplying ClinicalTrials.gov document identifiers in the 'NCTID' parameter when posting the request.

### Authorization
The instructions for getting your credentials and the API endpoint URL can be found under the section "Get Started" and "How to use the API" following this link: https://api-marketplace.work.iqvia.com/s/communityapi/a085w00000ytPu7AAE/external-api-marketplaceclinicaltrialsbiomarkers .

In [13]:
import getpass

# Get URL and credentials from customers
api_endpoint_url = input('Please enter the API URL: ').rstrip('/')

mkp_user = input("Marketplace clientId: ")
mkp_password = getpass.getpass("Marketplace clientSecret: ")
mkp_headers = {'clientId': mkp_user, 'clientSecret': mkp_password}

print("Thanks for inputting URL, your user name and password!")

Please enter the API URL: https://vt.eu-apim-devtest.solutions.iqvia.com/eu/clinicaltrials/api/v2/clinicaltrials/biomarkers
Marketplace clientId: c21726a44cf549dda5315fd8d35f467c
Marketplace clientSecret: ········
Thanks for inputting URL, your user name and password!


### Example: Make a request for a list of known Clinical Trials documents
Clinical Trials Biomarker NLP API expects ClinicalTrials.gov document identifier (NCTID) as input parameter. This example shows how to make a request to the API with a list of NCTIDs.


In [14]:
import requests
import time

# Define input NCTIDs
input_nctids = ["NCT04875351", "NCT03180086"]

# Make a request
print("Posting request to extract biomarkers and related diseases from specified CT.gov documents...")

response = requests.post(api_endpoint_url, headers=mkp_headers, json={"params": {"NCTID": input_nctids}, 'row_limit': 10})

# Poll the API until results are available
while response.status_code == 202:
    print('Results are not available yet. Waiting 10 seconds before polling again...')
    time.sleep(10)
    # Use the run id from the Post request to get results
    run_identifier = response.json()['id']    
    response = requests.get(url=f'{api_endpoint_url}/{run_identifier}', headers=mkp_headers)    
    
# Check the response
if response.status_code == 200:
    print("Success!")
    results_json = response.json()
else:
    raise Exception(f"Unexpected status code: {response.status_code}")

print(f'Results: \n{results_json}\n')

Posting request to extract biomarkers and related diseases from specified CT.gov documents...
Results are not available yet. Waiting 10 seconds before polling again...
Results are not available yet. Waiting 10 seconds before polling again...
Results are not available yet. Waiting 10 seconds before polling again...
Success!
Results: 
[{'doc_id': 'NCT04875351', 'results': [{'Biomarker': {'logical_column_id': 0, 'value': 'ERBB2', 'indexed_spans_outer': [[15358, 15362]], 'indexed_spans_inner': [[15358, 15362]], 'text_spans_outer': [[213, 217]], 'text_spans_inner': [[213, 217]]}, 'Relation': {'logical_column_id': 1, 'value': 'negative', 'indexed_spans_outer': [[15363, 15371]], 'indexed_spans_inner': [[15363, 15371]], 'text_spans_outer': [[218, 226]], 'text_spans_inner': [[218, 226]]}, '[SID] Diseases and Disorders': {'logical_column_id': 2, 'value': 'meddra', 'indexed_spans_outer': [[3419, 3432]], 'indexed_spans_inner': [[3419, 3432]], 'text_spans_outer': [[0, 13]], 'text_spans_inner': [[0,

Now that we have got the JSON responses from the Clinical Trials Biomarker API, we could convert the useful information associated with the keys into a pandas dataframe.

In [17]:
import pandas as pd

# initiate an empty dataframe
df = pd.DataFrame()
pd.set_option("display.max_rows", None, "display.max_columns", None, "display.width", 1000)

# Retrieve main results from the JSON response, please note this cell would fail if the request failed in the last step
for inner_results in results_json:
    for result_dict in inner_results['results']:
        df_dict = {}
        for key, value_dict in result_dict.items():                                                                                                                                                                                                                                                                                                                           
            df_dict[key] = value_dict['value']                                                                                                                                                                                                                                                                                                                                
        df_dict['Doc Id'] = inner_results['doc_id']                                                                                                                                                                                                                                                                                                                           
        df = pd.concat([df, pd.DataFrame.from_records([{**df_dict}])], ignore_index=True)                                                                                                                                                                                                                                                                                     

# Print the DataFrame                                                                                                                                                                                                                                                                                                                                                         
df

,Biomarker,Relation,[SID] Diseases and Disorders,[NID] Diseases and Disorders,[PT] Diseases and Disorders,Section,NCTID,text,Doc Id
0,ERBB2,negative,meddra,10006187,Breast cancer,Inclusion Criteria & Brief Title,NCT04875351,Breast Cancer Index (BCI) Registry ... Inclusi...,NCT04875351
1,ERBB2,negative,meddra,10006187,Breast cancer,Inclusion Criteria & Condition,NCT04875351,Breast Cancer... Inclusion Criteria: - Early s...,NCT04875351
2,ERBB2,negative,nlm_plus,D001943,Breast Neoplasms,Inclusion Criteria & Brief Title,NCT04875351,Breast Cancer Index (BCI) Registry ... Inclusi...,NCT04875351
3,ERBB2,negative,nlm_plus,D001943,Breast Neoplasms,Inclusion Criteria & Condition,NCT04875351,Breast Cancer... Inclusion Criteria: - Early s...,NCT04875351
4,ERBB2,positive,meddra,10006187,Breast cancer,Inclusion Criteria & Brief Title,NCT04875351,Breast Cancer Index (BCI) Registry ... Inclusi...,NCT04875351
5,ERBB2,positive,meddra,10006187,Breast cancer,Inclusion Criteria & Condition,NCT04875351,Breast Cancer... Inclusion Criteria: - Early s...,NCT04875351
6,ERBB2,positive,nlm_plus,D001943,Breast Neoplasms,Inclusion Criteria & Brief Title,NCT04875351,Breast Cancer Index (BCI) Registry ... Inclusi...,NCT04875351
7,ERBB2,positive,nlm_plus,D001943,Breast Neoplasms,Inclusion Criteria & Condition,NCT04875351,Breast Cancer... Inclusion Criteria: - Early s...,NCT04875351
8,ESR1,positive,meddra,10006187,Breast cancer,Inclusion Criteria & Brief Title,NCT04875351,Breast Cancer Index (BCI) Registry ... Inclusi...,NCT04875351
9,ESR1,positive,meddra,10006187,Breast cancer,Inclusion Criteria & Brief Title,NCT04875351,Breast Cancer Index (BCI) Registry ... Inclusi...,NCT04875351


### Optional: Make a request to ClinicalTrials.gov API to get NCTIDs associated with a specific disease

Search URL for the ClincialTrials.gov API that allows user to search for the NCTId's of 10 documents. Search can be customized by modifying the study_fields expr value. In this example, the search will return 10 NCTIds of clinical trials about Breast Cancer.

In [4]:
import json

# Search URL for the ClincialTrials.gov API that allows user to search for the NCTId's of 10 documents
# Search can be customized by modifying the study_fields expr value
# This search will return 10 NCTIds of clinical trials about Breast Cancer
url = "https://clinicaltrials.gov/api/query/study_fields?expr=Breast+Cancer&fields=NCTId&min_rnk=1&max_rnk=10&fmt=json"
response = requests.get(url)

# Read in the JSON from the ClinicalTrials.gov API request
j = json.loads(response.text)

# initialize list for storing the NCTId's we will submit to endpoint
nctids = []

# Pull out the NCTId's for each document returned by the ClinicalTrials.gov and add to the list
for item in j['StudyFieldsResponse']['StudyFields']:
    nctids.append(item['NCTId'][0])

print(nctids)

['NCT04875351', 'NCT03180086', 'NCT03662633', 'NCT02493569', 'NCT03598660', 'NCT04167605', 'NCT04516330', 'NCT04495244', 'NCT03343691', 'NCT05082740']


That's it! Hope you find this tutorial useful! Bye!